In [1]:
# Dependencies
import numpy as np
import tweepy
import time
import json
import pandas as pd
       
# Twitter API Keys
# consumer_key = "TEs2QmTsz4ylhWSqXWHOvVn99"
# consumer_secret = "yA5M95uhTdeAKLondYQ9LDMPR5LWKc6wX70Yry1oRDHWtBN7gg"
# access_token = "400199431-wb53H1jfbkDwY0xVS9aZbgWZrWlXcWkhXdFuBKMh"
# access_token_secret = "IRvQVov2vmopxjeTutR0jn57lnMlYtr8syHQrGbLgbmW9"

consumer_key = "PAmpYOq5LwscMnRSFBQqwNjxH"
consumer_secret = "uvvEfLcCoU9zBsRwc3wbWNGwUy7BUVtYH3376MHpLDy7oZ6qUK"
access_token = "907734237478080523-BAtPF5z8EzzwyPef0f04JC12LOQSCim"
access_token_secret = "0F5xIDmAwVAy7pQcOWs0mMFDedr9po1tLJuLNk8r3gMuG"

# Twitter Credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit = True, wait_on_rate_limit_notify = True)


In [2]:
# Get a list of locations in and outside the belt (in-belt, out-belt)
base_data = pd.read_csv("Zipcode_States_With_Geo_Cordinates.csv")
lat = base_data.groupby("State")["Latitude"].mean()
lon = base_data.groupby("State")["Longitude"].mean()
dfff = {"lat":[], "lon":[]}
for each in lat.index:
    dfff["lat"].append(lat[each])
    dfff["lon"].append(lon[each])
hah = pd.DataFrame(dfff)
base_data_state = hah.set_index([lat.index])

base_data_state.head(10)

,lat,lon
State,,
Alabama,32.886361,-86.813639
Alaska,61.456423,-152.486981
Arizona,33.610082,-111.647927
Arkansas,35.124723,-92.402676
California,36.170738,-119.792701
Colorado,39.264176,-105.317544
Connecticut,41.563130,-72.804928
Delaware,39.300372,-75.526776
District of Columbia,38.896231,-77.021235


In [3]:
base_data_state = base_data_state.reset_index()
base_data_state

,State,lat,lon
0,Alabama,32.886361,-86.813639
1,Alaska,61.456423,-152.486981
2,Arizona,33.610082,-111.647927
3,Arkansas,35.124723,-92.402676
4,California,36.170738,-119.792701
5,Colorado,39.264176,-105.317544
6,Connecticut,41.563130,-72.804928
7,Delaware,39.300372,-75.526776
8,District of Columbia,38.896231,-77.021235
9,Florida,28.129716,-81.956165


In [4]:
tweet_df = {}
tweet_df = pd.DataFrame(tweet_df)
tweet_df["state"] = ""
tweet_df["text"] = ""
tweet_df["lat"] = ""
tweet_df["lon"] = ""
tweet_df["followers"] = ""
tweet_df["favorites"] = ""
tweet_df["friends"] = ""
tweet_df["statuses_count"] = ""
tweet_df["compound"] = ""
tweet_df["pos"] = ""
tweet_df["neg"] = ""


tweet_df.head()


,state,text,lat,lon,followers,favorites,friends,statuses_count,compound,pos,neg


In [5]:
# Loop through DataFrame for each Latitude and Longitude
k = 0
for index, row in base_data_state.iterrows():    
    
    # Get Tweets for all locations
    location = "%s,%s,100mi" % (row["lat"], row["lon"])     
    public_tweets = api.search("", geocode = location, count=100)
    
    # Extract all hashtags from tweets directed to the account
    for tweet in public_tweets["statuses"]:

        # Print tweet in JSON
        # print(json.dumps(tweet, sort_keys=True, indent=4, separators=(',', ': ')))

        # Print the Tweet Text         
        print(tweet["text"])

        # Print tweet text
        tweet_df.set_value(k, "state", row["State"])
        tweet_df.set_value(k, "text", tweet["text"])
        tweet_df.set_value(k, "lat", row["lat"])
        tweet_df.set_value(k, "lon", row["lon"])

        tweet_df.set_value(k, "followers", tweet["user"]["followers_count"])
        tweet_df.set_value(k, "favorites", tweet["user"]["favourites_count"])
        tweet_df.set_value(k, "friends", tweet["user"]["friends_count"])
        tweet_df.set_value(k, "statuses_count", tweet["user"]["statuses_count"])

        # Save to a csv for every state
        tweet_df.to_csv("Tweet_Dump_5.csv")
        
        k = k + 1
         

RT @_MauroGil: @JornalOGlobo @OGlobo_Cultura "CONTROLE DE ARMAS!"

"Enquanto isso, vou passear com meus seguranças armados" https://t.co/qV…
@ClintSmithIII @AlexGeorge5 If you hate it so bad, then move to North Korea or somewhere far away. I can assure you… https://t.co/iPvbVF830X
RT @ErinDionne1: One nigga can fuck it up for everybody 🙄
one person followed me and one person unfollowed me // automatically checked by https://t.co/hLnhTul0QJ
Nah that’s just what u call a stress free relationship https://t.co/cwB0DpFnN7
Recording shit at 2k to render in 720p
Hell yeah
now is when we need prayer more than ever. my prayers go out to Las Vegas.💗
RT @Dwm923: Merch https://t.co/wmuyO4Y19B
RT @thatblondebishh: And every single time you break a girls heart in two, there’s a little less of it when it comes back to you..
Finally getting around to watching Icarus. If you are an athlete, and haven’t watched it yet, you aren’t an athlete.
@NobodyButJackie 💙💙💙💙
RT @JerryHinnen: Me, on a thing that mak

@byunnina_ คนที่กูบอกมึง แต่เป็นใครก็ไม่รู้555
Kok cepettt https://t.co/iEXKF5vgQg
@munfess Beruntung bat si karin
@askmenfess Megang bh? Beha? Megang tete km? Woy gmn sih ga ngerti bagian itu 

Tinju ae tinjuuu
@jangjunable Rasanya gimana pas sementara di operasi? Sakit ga? Perih? Ato gmna dah penasaran 😂
@jangjunable Semoga tahun depan gue sanggup ga megang hp-_- rasanya pas di operasi gimna? Terus operasinya wkt itu… https://t.co/whIy8mmDf5
@jangjunable Sblm operasi brp?
@askmenfess Gue selasa
Kembali lagi ga sih? https://t.co/3anhzKzjXY
@askmenfess Gue taun depan tapi😂 blm cukup umur dilarang dokter
@rlthingy 2
@askmenfess HEMAT
อยากกลับบ้าน แง
ทำขบวนตัดแต่โฟม ปวดมือเด้อ
เหนื่อย
@munfess Matot
Kalau pacaran sm om om tentara termaksud sugar daddy bukan sih:( bodo ah tolol https://t.co/LnnGfno3Dx
RT @yluvmac: END!
1.5 magnitude #earthquake. 71 km from Willow, AK, #UnitedStates https://t.co/Pp9lKofEzk
USGS reports a M1.5 #earthquake 71km WNW of Willow, Alaska on 10/1/17 @ 16:30:35 UTC

RT @Aetius_stronk: https://t.co/AgWluCW0qT
Can you recommend anyone for this #job? MEDICAL RECEPTIONIST PN - PD's - https://t.co/09M2YfgFB0 #Healthcare #Phoenix, AZ #Hiring
update on elder anderson‼️ only 7 short months left till he comes home!!! he wants new people to email him so hcand… https://t.co/z0dqurl9TY
@R_Byrnee @AmiiM_ @obamaout12 @KEEMSTAR Ok so picture this. I break into your house and you’re across the room from… https://t.co/fOjMoOMijT
Cause they don't care now it's cool so they don't wanna be left out lmao https://t.co/hISopwFuNP
@bradshoemaker Totally numb to it at this point, everyday feels the same
RT @CaptainsLog2017: Sarah Sanders says more facts needed before discussing #GunControlNow

Over 50 dead
500+ injured
Due to automatic gunf…
RT @francineleroy: @thismyshow @benshapiro  https://t.co/2kmTPkR7DT
RT @imOliviaa_: Treys advice (Vine by @theregaljoe) https://t.co/hKsidiZaiZ
RT @CaptainsLog2017: Sarah Sanders says more facts needed before discussing #GunControlNow

Los Angeles police deny confirming Tom Petty's death to CBS https://t.co/stNZc0SGwn
@Mr_Will_I_am34 Lmaooooo
More Questions Than Answers at Dior - The Business of Fashion https://t.co/gOfc3atCGB #Hautecouture #Fashion
RT @DreGatsby: I didn’t live up to my full potential as a baby 😂 https://t.co/aX8zDvMOjq
RT @awkwardsmol: que es eso de decir "no me gusta el anime" hay como 48385828 generos y todos te desagradan? no entiendo
Balenciaga Faces Fresh Casting Controversy at Paris Fashion Week - WWD https://t.co/mCsMOaajwQ #Hautecouture #Fashion
MAÑANA VOY A IR A COMPRAR LA ENTRADA CREO QUE VOY A MORIRME
RT @chipacite: hola no es mal momento para recordar que extraño a los pilotos o si
Vacío vacío el luna park
Chanel Iman catches the eye in thigh-skimming satin minidress... while Jourdan Dunn sports frayed denim as they lead ... - Daily Mail …
Why am I so goofy
Hero. https://t.co/UhuRY4nWCZ
@AW_ThatsBria I seen one girl snap on a nigga for that I was shaking my head 😂
Fashion forward! Actres

https://t.co/Drzptb8vHz
RT @_Loblolly: ⚡️ “Democrats: Know Which Side You're On” by @SallyAlbright

https://t.co/Drzptb8vHz
How to guarantee that #mtv #trl flops before it starts? Have show runner #albertlewitinn show his ignorance in an interview. Far from #woke.
@AndieNicoleeee We get it I fucked your friend and to be honest I'm probably gonna do it again so 🤷🏽‍♂️🤷🏽‍♂️
Askholes, we all have 'em in our lives https://t.co/y0UbanGV8q
RT @hoesuueee: https://t.co/QTIHzQGpmb
I love The Twilight Zone so much https://t.co/a3293LRYYt
@marcusisill But I see another Tony Romo in Oakland..? lol Vice Versa my man.. Without Carr you guys are in the Same Boat. Suck in the 90's.
You're a good man, Joe. :-) https://t.co/ZBs9jmWpVG
RT @thestorysofarca: Today is both heartbreaking and hard to stomach. Extend love to those around you today, please.
welp 🤦🏽‍♀️ https://t.co/lDFB6ztgDT
RT @MBARI_News: Bathydoris aioca, a #deepsea nudibranch in the Gulf of California at 3,179 m https://t.co/ID08ODkJ5S #MBAR

...

RT @BarackObama: Michelle &amp; I are praying for the victims in Las Vegas. Our thoughts are with their families &amp; everyone enduring another se…
RT @FBI: Correction: Please call 1-800-CALL-FBI (225-5324) with videos &amp; photos from the Las Vegas shooting on 10/1/17. https://t.co/DdvUSi…
@ANTITRUMPMVMT @ObamaMama107 Disgusting, irresponsible action &amp; assumptions by the NRA. Got to do that advertising… https://t.co/zMLLxfggQp
RT @byHeatherLong: Jonathan Smith, 30, saved ~30 people last night before he was shot in the neck. He might live w/the bullet for rest of h…
If the point of a Tinder bio is to scare people away, then I am really good at Tindering.
RT @byHeatherLong: Jonathan Smith, 30, saved ~30 people last night before he was shot in the neck. He might live w/the bullet for rest of h…
RT @JuddLegum: Actually he fits the profile pretty well. Most mass shooters are white men. https://t.co/WRnUR1acFB
@KatjaMagyarics most of those in FL are single shootings.... This is the od

RT @NSOME_: เป็นป้ะ ตอนไม่มีโมเม้นล่ะชงเก่งคิดแคปชั่นเก่ง แต่พอโมเม้นมาแล้วเหมือนสมองเบลอหูอื้อตาลายคุมสติตัวเองพิมพ์ให้ถูกยังยากรเยบพฮือ แ…
RT @BR_DougFarrar: Keenum needs defined openings to succeed deep. Had them vs. Bucs. Didn't against Lions. Detroit did a great job matching…
RT @OriginalFunko: RT &amp; follow @OriginalFunko for the chance to win an #NYCC 2017 exclusive Krombopulos Michael Pop! https://t.co/UNSazUEbtx
RT @MRHS_CC: #SaveTheDate College App Family Night October 5th!! #PersonalStatements #FinancialAid #GetStartedNow https://t.co/AKPrWGa6Gi
@carrieksada @RuthieRedSox @sethmoulton That's why they're illegal in 50 states, but can you prevent one of those i… https://t.co/KCMh4tjEGo
@FlyinBrianJ Only 1st choice for egg salad macaroni salad and potato salad.
Starting a new business and trying to grow your brand can be scary. https://t.co/TmYjHYDrL4 https://t.co/l1dexG2aLx
Falls and Older Adults https://t.co/ROVJ8QIc2X #alz,#caregiver
Gun control isn't a "solution" to what 

RT @CW_Arrow: Feed your belly this super season with new episodes of your favorite heroes, starting Monday, October 9 on The CW. #Arrow htt…
RT @CW_Arrow: Feed your belly this super season with new episodes of your favorite heroes, starting Monday, October 9 on The CW. #Arrow htt…
RT @CW_Arrow: Feed your belly this super season with new episodes of your favorite heroes, starting Monday, October 9 on The CW. #Arrow htt…
RT @CW_Arrow: Feed your belly this super season with new episodes of your favorite heroes, starting Monday, October 9 on The CW. #Arrow htt…
RT @CW_Arrow: Feed your belly this super season with new episodes of your favorite heroes, starting Monday, October 9 on The CW. #Arrow htt…
RT @CW_Arrow: Feed your belly this super season with new episodes of your favorite heroes, starting Monday, October 9 on The CW. #Arrow htt…
@Bat_Cheeks Wish I could give ya a hug or help out some way. Let me know if there's anything you need.
this sickness needs to go away bc i really can’t af

@Camusdono *blush
Tune into 105.7 the FAN if you are located in Milwaukee to have a listen! https://t.co/9uoVh2iO9E
Scaly boys showing their buns~ 💚💙 (Art courtesy of @SteenExperiment) https://t.co/DWjZaoiwR1
@goodoldcatchy Our version of healthcare Obamacare makes sure you starve.  It's a catch 22.  Fined for not having i… https://t.co/hWcMfcKfgU
Hoping Justice Kennedy continues to show fairness on this issue. https://t.co/J1f3emFXop
#EspnLeague fala do packao
RT @jrsalzman: There is no way this guy did all this without someone noticing. People around him are lying. https://t.co/LHQ1BsMRbr
@traceyfanclub Ikr, I too would like to kick myself if i paid 15$ for a churro. They good tho. Theyre real good.
@DineshDSouza she was old news before she opened her mouth
*Absolutely hope my spelling issues dont degrade the message.
3800 Gang Members Charged in Transnational DOJ Operation
https://t.co/zGcgdpclaJ
i may have a problem https://t.co/j10wgsVNI9
@ShortSleeveSuit @dave_cactus Exhibit A
ht

Do whatever you do to bring positive energy into the world - pray, light a candle, sing a song, recite a poem, bang…https://t.co/QeK7gCagZx
@debilynnking @michaelianblack @FrankConniff Well you already need extreme licensing to own one.
today's stats : 2 new followers, 8 new unfollowers, 2109 non-followers. found via https://t.co/qlZLGPMYcb
#21savage and #amberrose react to #slutwalk https://t.co/yLLI5Tm6T1
RT @tannerhuels: This CBS executive has "no sympathy" for the Las Vegas victims because most are probably Republican. 

Fire. Her. Now. @CB…
@edisn @BoycottBertha @VoiceOfReasonNW @TJMastersson @Anchower @NiHauMau @uncloudytweet @jamesvicktory @PotPolitics… https://t.co/MrWMg0IAZE
You know, when you tell people something and they act uninterested, it just means your suppose to keep your mouth shut....
@MarkSKrikorian A 1% tax on these remittances would help a lot to reduce debt
RT @DianaRambles: Chocolate Peanut Butter Pudding Pie made with Oreo O's #ad #BestCerealEver
https://t.co/

In [6]:
# We're going to store the tweets inside a dataframe associated with the state, location, and in-belt/out-belt 

In [7]:
# We're going to process all the tweets

In [8]:
# We're going to group the data in-belt/out-belt